In [9]:
import math
import os
import shutil

In [10]:
def parse_problem(problem_file):

    x_obs = []
    t_obs = []
    on = False

    with open(problem_file, "r") as f:
        for line in f.readlines():
            if "(x)" in line:
                init_x = float(line.strip().replace("(", "").replace(")", "").split(" ")[2])
            elif "(on)" in line:
                on = True
            elif "(time_obs" in line:
                t_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]
            elif "(x_obs" in line:
                x_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]


    observations = list(zip(x_obs, t_obs))
    
    return observations, init_x, on

In [11]:
def define_ha(observations, on, init_x):
    
    num_obs = len(observations)
    
    ha_str = ""
    
    # Variables
    ha_str += "[0,{}] time;\n".format(observations[-1][1] + 0.05)
    ha_str += "[{},{}] x;\n".format(17,23)
    ha_str += "[0,{}] clock;\n".format(observations[-1][1] + 0.05)
    ha_str += "\n"
    
    # Modes
    modes = {}
    num_modes = 0
    for hmode in ["on", "off"]:
        for obs_index in range(num_obs+1):
            modes[(hmode,obs_index)] = num_modes + 1
            num_modes += 1
    
    for hmode in ["on", "off"]:
        if hmode == "on":
            inv = "<= 22.0"
            flow_x = "0.8"
            jump_condition = "> 21.0"
            dst_offset = num_obs + 1
        else:
            inv = ">= 18.0"
            flow_x = "-0.2"
            jump_condition = "< 19.0"
            dst_offset = -num_obs - 1 
            
        for obs_index in range(num_obs+1):
            mode_id = modes[(hmode, obs_index)]
            ha_str += "// {}_obs{}\n".format(hmode,obs_index)
            ha_str += "{\n"
            ha_str += "\t mode {};\n".format(mode_id)
            # Invariant
            ha_str += "\t invt:\n"
            if obs_index < num_obs:
                ha_str += "\t\t (and (x {}) (clock < {} + 0.05));\n".format(inv, observations[obs_index][1])
            else:
                ha_str += "\t\t (and (x {}) (clock < {} + 0.05));\n".format(inv, observations[-1][1])
            # Flow
            ha_str += "\t flow:\n"
            ha_str += "\t\t d/dt[x] = {};\n".format(flow_x)
            ha_str += "\t\t d/dt[clock] = 1.0;\n"
            
            # Jumps
            
            ha_str += "\t jump:\n"
            ha_str += "\t\t (x {}) ==> @{}(and (x' = x) (clock' = clock));\n".format(jump_condition, mode_id + dst_offset)
            if obs_index < num_obs:    
                ha_str += "\n"
                ha_str += "\t\t (and (and (clock > {clock_obs} - 0.05) (clock < {clock_obs} + 0.05)) (and (x > {x_obs} - 0.1) (x < {x_obs} + 0.1))) ==> @{dst_id}(and (x' = x) (clock' = clock));\n".format(clock_obs = observations[obs_index][1], x_obs = observations[obs_index][0], dst_id = mode_id+1)
                
            ha_str += "}\n"
            
    # Init
    if on:
        init_mode = modes[("on", 0)]
    else:
        init_mode = modes[("off", 0)]
    ha_str += "init:\n"
    ha_str += "@{} (and (x = {}) (clock = 0.0));\n".format(init_mode, init_x)
    ha_str += "\n"
    
    # Goal
    ha_str += "goal:\n"
    for hmode in ["on", "off"]:
        mode_id = modes[(hmode, num_obs)]
        ha_str += "@{} true;\n".format(mode_id)
            
    return ha_str   

In [13]:
num_instances = 10
horizons = [2, 4, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

dfolder = "benchmarks/thermostat/dreach"
if os.path.exists(dfolder):
    shutil.rmtree(dfolder)
os.makedirs(dfolder)


for pnum in range(num_instances):
    for h in horizons:
        pname = 'inst_{pnum}_{h}.drh'.format(pnum=pnum, h=h)
        
        observations, init_x, on = parse_problem("benchmarks/thermostat/generation/inst_{}_{}_5_100.pddl".format(pnum, h))

        ha_str = define_ha(observations, on, init_x)
        with open(dfolder + '/' + pname, 'w') as f:
            f.write(ha_str)